# 빅프9조

 필수 라이브러리 import

In [2]:
import numpy as np
import pandas as pd

from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time

import folium

파일경로, csv인코딩 설정

In [3]:
data_path = "전처리완료데이터/"
raw_path = "전처리필요데이터/"
encoding = "cp949"
extension = ".csv"

경찰서 목록

In [4]:
police = np.array([37.679699, 127.055066], dtype=np.float64)

격자점 생성에 필요한 상수 선언

In [5]:
NS_DIFF_1M = 0.000007
EW_DIFF_1M = 0.000009
PATROL_HR = 900
GRID_DIFF = 90

# 기본 격자점 생성

원형으로 필터링된 격자점 반환

In [6]:
def filtering(police, grid):
    ft = (grid - police).copy()
    ft[:,0] /= NS_DIFF_1M
    ft[:,1] /= EW_DIFF_1M
    ft = np.sqrt( np.sum( np.square(ft), axis=1 ))
    grid = grid[ft <= PATROL_HR + 10, :]
    return grid

def grids(police):
    hcount = PATROL_HR//GRID_DIFF
    ew = np.arange(-hcount, hcount + 1, dtype=np.float64)
    ns = ew.reshape(-1,1).copy()
    ew *= EW_DIFF_1M * GRID_DIFF
    ns *= NS_DIFF_1M * GRID_DIFF
    
    result = np.full((2*hcount+1, 2*hcount+1, 2), police, dtype=np.float64)
    result[:,:,0] += ns
    result[:,:,1] += ew
    
    return pd.DataFrame(filtering(police, result.reshape(-1,2)), columns=["위도", "경도"])

지도에 해당 위도경도에 위치한 점 찍기

In [7]:
def draw_marker(m, grid, color):
    color = "#" + color
    for i in grid.values:
        folium.CircleMarker(location=i, radius=5, color=color).add_to(m)

지도객체 생성, 경찰서 특별 표식

In [11]:
m = folium.Map(location = police, zoom_start=60)
folium.Marker(location=police, 
              popup='지구대',
              icon=folium.Icon(color='lightblue', icon='star')
).add_to(m)

기본 격자점 생성

In [12]:
police_grid = grids(police)

기본 격자점 그리기

In [13]:
draw_marker(m, police_grid, color='000000')
m

# 샘플링

뽑을 개수 (5개)

In [14]:
SAMPLING = 5

격자점 범위 내에서 무작위로 5개 점 추가

In [15]:
def sampled_grids(grid):
    interval_size = SAMPLING+1
    result = np.zeros((grid.shape[0]*interval_size, 2), dtype=np.float64)
    for i in range(0, result.shape[0], interval_size):
        ran = np.zeros((interval_size, 2), dtype=np.float64)
        ran[1:] = np.random.uniform(-1, 1, (interval_size-1, 2))
        ran[:, 0] *= (GRID_DIFF/2)*NS_DIFF_1M
        ran[:, 1] *= (GRID_DIFF/2)*EW_DIFF_1M
        ran += grid.values[i//interval_size]
        result[i:i+interval_size] += ran
    return pd.DataFrame(result, columns = ["위도", "경도"])

추가한 데이터 csv내보내기

In [16]:
police_sampled = sampled_grids(police_grid)
police_sampled

,위도,경도
0,37.673399,127.054256
1,37.673226,127.053969
2,37.673140,127.054040
3,37.673158,127.053900
4,37.673174,127.054229
...,...,...
1945,37.686223,127.055503
1946,37.685906,127.055661
1947,37.685822,127.055937
1948,37.686147,127.055589


In [17]:
police_sampled.to_csv(raw_path + "sampled_grid" + extension, index=False, encoding=encoding)

### 주소로 변환

https://www.geoservice.co.kr/

내보낸 csv 파일 이용, 위 링크에서 역지오코딩, 주소얻기

주소 기본 데이터 생성

In [23]:
def addr_data(file_path):
    data=pd.read_csv(file_path)
    data["이름"] = None
    data["확률"] = None
    data["index"]=[i//(SAMPLING+1) for i in range(data.shape[0])]
    return data

In [31]:
police_addr = addr_data(data_path + "sampled_addr.csv")
police_addr

,위도,경도,지번주소,도로명주소,이름,확률,index
0,37.673399,127.054256,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None,0
1,37.673226,127.053969,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None,0
2,37.673140,127.054040,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None,0
3,37.673158,127.053899,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None,0
4,37.673174,127.054229,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None,0
...,...,...,...,...,...,...,...
1945,37.686223,127.055503,서울특별시 노원구 상계동 1202-1,서울특별시 노원구 동일로 1772,None,None,324
1946,37.685906,127.055661,서울특별시 노원구 상계동 1202-1,서울특별시 노원구 동일로 1772,None,None,324
1947,37.685822,127.055937,서울특별시 노원구 상계동 산 12-1,서울특별시 노원구 동일로 1772,None,None,324
1948,37.686147,127.055588,서울특별시 노원구 상계동 1202-1,서울특별시 노원구 동일로 1772,None,None,324


# 건물 이름 크롤링

지연 시간(1초), 웹 지도 url(카카오맵) 설정

In [25]:
DELAY_TIME = 0.5
map_url = "https://map.kakao.com/"

브라우저 드라이버 반환(no 전달시 창 안뜸)

In [26]:
def openweb(mode="window"):
    if mode == "no":
        options = wb.ChromeOptions()
        options.add_argument('headless')
        driver = wb.Chrome(options=options)
    else:
        driver = wb.Chrome()
    driver.get(map_url)
    time.sleep(DELAY_TIME)
    return driver

검색어 입력 후 실행

In [27]:
def search(browser, address):
    search_box = browser.find_element(By.ID, "search.keyword.query")
    while search_box.get_attribute("value"):
        search_box.send_keys(Keys.CONTROL, 'a')
        search_box.send_keys(Keys.BACKSPACE)
    search_box.send_keys(address)
    search_box.send_keys(Keys.RETURN)
    time.sleep(DELAY_TIME)

### 크롤링 함수

도로나 강 야외의 경우 지번주소는 있지만 우편번호가 없다

지번주소 먼저 검색 -> 야외 분류

우편번호 있으면 -> 도로명주소 검색

건물이름 뜨면 -> 이름 반환

안뜨면 -> 우편번호 반환

In [28]:
def find_name(browser, addr, road_addr):
    if "산" in addr:
        return "산"
    if "지하" in road_addr:
        return "지하철"
    search(browser, addr)
    try:
        postage = browser.find_element(By.CLASS_NAME, "zip")
    except NoSuchElementException:
        return "nozip"
    else:
        search(browser, road_addr)
        try:
            buildname = browser.find_element(By.CLASS_NAME, "building")
        except NoSuchElementException:
            return postage.text
        else:
            return buildname.text

주소 전부 검색 후 csv로 내보내기

In [30]:
def search_to_csv(browser, data, file_path):
    for i in range(len(data.index)):
        data.iloc[i, 4] = find_name(browser, data.iloc[i, 2], data.iloc[i, 3])
    data.to_csv(file_path, index=False, encoding=encoding)

In [32]:
browser = openweb("no")
search_to_csv(browser, police_addr, data_path+"sampled_name.csv")

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome-headless-shell=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00007FF641FF1F52+60322]
	(No symbol) [0x00007FF641F6CEC9]
	(No symbol) [0x00007FF641E27EBA]
	(No symbol) [0x00007FF641E1070D]
	(No symbol) [0x00007FF641E10410]
	(No symbol) [0x00007FF641E0E39D]
	(No symbol) [0x00007FF641E0EBAF]
	(No symbol) [0x00007FF641E2D7E3]
	(No symbol) [0x00007FF641E30641]
	(No symbol) [0x00007FF641E306E0]
	(No symbol) [0x00007FF641E6EDFF]
	(No symbol) [0x00007FF641E9C21A]
	(No symbol) [0x00007FF641E6ADB6]
	(No symbol) [0x00007FF641E9C430]
	(No symbol) [0x00007FF641EBBC80]
	(No symbol) [0x00007FF641E9BFC3]
	(No symbol) [0x00007FF641E69617]
	(No symbol) [0x00007FF641E6A211]
	GetHandleVerifier [0x00007FF6423094AD+3301629]
	GetHandleVerifier [0x00007FF6423536D3+3605283]
	GetHandleVerifier [0x00007FF642349450+3563680]


### 이름을 토대로 유형 분류, 확률 점수 부여

In [ ]:
police_per_data = pd.read_csv(data_path + "sampled_name.csv", encoding=encoding)
set(police_per_data["이름"])

장소 분류에 따른 범죄확률

In [ ]:
crime_per = pd.read_csv(data_path+"processed_crime.csv", encoding=encoding)
crime_per

분류 결과를 바탕으로 이름이 어떤유형인지 분류

자료에 맞춰 확률 계산후 대입

In [ ]:
def namespace(file_path):
    space = dict()
    with open(file_path, 'r', encoding='UTF8') as file:
        for i in file:
            temp=i.split(" - ")
            space[temp[0]] = temp[1].strip()
    return space

def insert_percentage(file_path, addr_data):
    space = namespace(file_path)
    for i in space:
        addr_data.loc[addr_data["이름"]==i, "확률"] = crime_per.loc[0, space[i]]

In [ ]:
insert_percentage(data_path + "이름유형분류.txt", police_per_data)       
police_per_data

동일 지역에서 샘플된 데이터끼리 확률 합 구하고 격자점 데이터에 새로운 열로 추가

In [ ]:
def insert_score(grid, percentage_data):
    score = percentage_data[["index", "확률"]].groupby("index").sum()
    grid["점수"] = score

In [ ]:
insert_score(police_grid, police_per_data)
police_grid

시각화를 위해 점수를 -255(초록)~255(빨강)으로 변환

변환후 rgb코드에 맞게 문자열 생성 안전->초록, 위험->빨강

In [ ]:
def colorstr(num):
    b = num>=0
    num=hex(int(255-abs(num)))[2:]
    if len(num)==1:
        num="0"+num
    return "#ff"+num*2 if b else "#"+num+"ff"+num

In [ ]:
def score_to_color(grid):
    result = grid["점수"].copy()
    mins = result.min()
    maxs = result.max()
    result = ((result - mins) / (maxs - mins) * 2 - 1) * 255
    result = result.apply(colorstr)
    return result

In [ ]:
def draw_score(grid, color):
    for i in grid.index:
        folium.CircleMarker(location=grid.iloc[i,:2].values,
                            radius=20,
                            color=color[i],
                            fill=True).add_to(m)

장소에 따른 위험도 시각화

In [ ]:
police_grid_color = score_to_color(police_grid)
police_grid_color

In [ ]:
draw_score(police_grid, police_grid_color)
m

해당 범위 안에 목표물이 몇개 있는가?

In [ ]:
#격자점반경에 대해
def location_count(grid, location):
    temp = (grid - location).copy()
    temp.iloc[:,0] /= NS_DIFF_1M
    temp.iloc[:,1] /= EW_DIFF_1M
    temp = np.sqrt( np.sum( np.square(temp), axis=1 ))
    return len(grid.loc[temp <= GRID_DIFF/2, :].index)

#순찰반경에 대해
def police_count(grid, police):
    temp = (grid - police).copy()
    temp.iloc[:,0] /= NS_DIFF_1M
    temp.iloc[:,1] /= EW_DIFF_1M
    temp = np.sqrt( np.sum( np.square(temp), axis=1 ))
    return len(grid.loc[temp <= PATROL_HR+GRID_DIFF/2, :].index)

### 안심시설물 데이터 이용

In [ ]:
ansim_data = pd.read_csv(data_path+"processed_ansim.csv", encoding=encoding)
ansim_data

In [ ]:
#draw_marker(m, ansim_data, color='0000ff')
m

### cctv 데이터 이용

In [ ]:
cctv = pd.read_csv(data_path+"processed_cctv.csv", encoding=encoding)
cctv

In [ ]:
#draw_marker(m, cctv, color='00ffff')
m

### 보안등 데이터 이용

보안등은 너무 많아서 순찰반경내의 것만 잘라 사용

In [ ]:
def light_cut(police, light_data):
    NS_MAX=police[0]+(PATROL_HR+GRID_DIFF/2)*NS_DIFF_1M
    NS_MIN=police[0]-(PATROL_HR+GRID_DIFF/2)*NS_DIFF_1M
    EW_MAX=police[1]+(PATROL_HR+GRID_DIFF/2)*EW_DIFF_1M
    EW_MIN=police[1]-(PATROL_HR+GRID_DIFF/2)*EW_DIFF_1M
    return light_data.loc[(NS_MIN<=light_data["위도"])&(light_data["위도"]<=NS_MAX)&
                           (EW_MIN<=light_data["경도"])&(light_data["경도"]<=EW_MAX),:]

In [ ]:
light_data = pd.read_csv(data_path+"processed_light.csv", encoding=encoding)
light_data

In [ ]:
light_police = light_cut(police, light_data)
light_police

In [ ]:
#draw_marker(m, light_police, color='ffff00')
m